In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime 
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly


mpl.rcParams['figure.figsize']=(16,9)
pd.set_option('display.max_rows', 500)
sns.set(style="darkgrid")

import plotly.graph_objects as go

# Data Preparation

1. focus is always to understand the final data structure
2. support each step by visual analytics

## Johns Hopkins GITHUB csv data

In [ ]:
datapath= 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [ ]:
pd_raw.head()

In [ ]:
pd_raw.columns[:]

In [ ]:
time_idx=pd_raw.columns[4:]

In [ ]:
df_plot= pd.DataFrame({ 'Date':
                       time_idx
    
})
df_plot.head()

In [ ]:
pd_raw['Country/Region'].unique()

In [ ]:
pd_raw['Country/Region']

In [ ]:
pd_raw[pd_raw['Country/Region']=='US']

In [ ]:
import numpy as np
country_list=['Italy', 'US', 'Spain','Germany', 'Korea,South']

In [ ]:
for each in country_list:
    df_plot[each]=np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))

In [ ]:
df_plot

In [ ]:
df_plot.set_index('Date').plot()

# Conversion of Date Objects

In [ ]:
df_plot.head()

In [ ]:
from datetime import datetime

In [ ]:
df_plot.Date[0]

In [ ]:
datetime.strptime(df_plot.Date[0], '%m/%d/%y')

In [ ]:
time_idx=[datetime.strptime(each, '%m/%d/%y') for each in df_plot.Date]
time_str=[each.strftime('%Y-%m-%d') for each in time_idx]

In [ ]:
time_idx

In [ ]:
df_plot['Date']=time_idx
type(df_plot['Date'][0])

In [ ]:
df_plot.head()

In [ ]:
df_plot.to_csv('processed.csv', sep=';', index=False)

In [ ]:
df_plot

# Relational data model - defining a primary key

In the relational model of databases, a primary key is a specific choice of a minimal set of attributes (columns) that uniquely specify a tuple (row) in a relation (table) (source: wiki)

A primary key’s main features are:

It must contain a unique value for each row of data.
It cannot contain null values.

In [ ]:
import pandas as pd

In [ ]:
datapath= 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [ ]:
pd_raw.head()

In [ ]:
pd_data_base=pd_raw.rename(columns={'Country/Region':'country','Province/State':'state' })

In [ ]:
pd_data_base

In [ ]:
pd_data_base=pd_data_base.drop(['Lat', 'Long'], axis=1)

In [ ]:
pd_data_base.head()

In [ ]:
pd_data_base.set_index(['state','country'])

In [ ]:
test_pd=pd_data_base.set_index(['state','country']).T
test_pd

In [ ]:
test_pd.columns

In [ ]:
test_pd.stack(level=[0,1]).reset_index()

In [ ]:
pd_relational_model=pd_data_base.set_index(['state', 'country'])\
.T \
.stack(level=[0,1]) \
.reset_index() \
.rename(columns={'level_0':'date', 0:'confirmed'},
       )

In [ ]:
pd_relational_model.head()

In [ ]:
pd_relational_model.dtypes

In [ ]:
pd_relational_model.to_csv('COVID_relational_confirmed.csv', sep = ';', index = None)

In [ ]:
print(pd_relational_model[pd_relational_model['country']=='US'].tail())

# Attention due to the new structure of the Johns Hopkins data set the state information US is in a second file

In [ ]:
data_path='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

In [ ]:
pd_raw_US=pd.read_csv(data_path)
pd_raw_US.head()

In [ ]:
# drop all non relevant fields! 
# Many of this fields do not belong to 
# a transactional data. These fields belong to so called master data
pd_raw_US=pd_raw_US.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','FIPS', 'Admin2',  'Lat', 'Long_', 'Combined_Key'],axis=1)
pd_data_base_US=pd_raw_US.rename(columns={'Province_State':'state'}).copy()

In [ ]:
## the stack command has problems when only one information is available within a stacking level
pd_relational_model_US=pd_data_base_US.set_index(['state']) \
                                .T                              \
                                .stack()             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
pd_relational_model_US['country']='US'
pd_relational_model_US['date']=[datetime.strptime( each,"%m/%d/%y") for each in pd_relational_model_US.date] # convert to datetime
pd_relational_model_US.head()

In [ ]:
# update the large relational file
pd_relational_model_all=pd_relational_model_US[pd_relational_model_US['country']!='US'].reset_index(drop=True)
pd_relational_model_all=pd.concat([pd_relational_model_all,pd_relational_model_US],ignore_index=True)

In [ ]:
pd_relational_model_all.to_csv('COVID_relational_confirmed.csv',sep=';',index=False)

In [ ]:
print(pd_relational_model_all[pd_relational_model_all['country']=='US'])

# cross check on full sum US data

In [ ]:
check_US=pd_relational_model_all[pd_relational_model_all['country']=='US']

In [ ]:
check_US[['date','country','confirmed']].groupby(['date','country']).sum()